In [1]:
import turicreate as tc
products = tc.SFrame('amazon_baby.sframe')

#ignore all 3*  reviews
products = products[products['rating']!= 3]

#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [2]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [3]:
products['word_count'] = tc.text_analytics.count_words(products['review'])

In [4]:
def TakeApartWordCount(dict, word):
    if word in dict.keys():
        return dict[word]
    else:
        return 0

In [5]:
for word in selected_words:
    products[word]=products['word_count'].apply(lambda x : TakeApartWordCount(x,word))

In [6]:
products

name,review,rating,sentiment,word_count,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,1,"{'recommend': 1.0,'disappointed': 1.0, ...",0.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,1,"{'quilt': 1.0, 'the':1.0, 'than': 1.0, 'fu ...",0.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,1,"{'tool': 1.0, 'clever':1.0, 'binky': 2.0, ...",0.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,1,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",0.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,1,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",0.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,1,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,1,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,1,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,1,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,1,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",0.0


In [7]:
max_usage_rate = 0
most_used_word = ''
min_usage_rate = 10^5000
min_used_word = ''
for word in selected_words:
    x = products[word].sum()
    if x>max_usage_rate:
        max_usage_rate = x
        most_used_word = word
    elif x<min_usage_rate:
        min_usage_rate = x
        min_used_word = word
print(most_used_word, max_usage_rate)
print(min_used_word, min_usage_rate)

great 55791.0
wow 425


In [8]:
train_data,test_data = products.random_split(.8, seed=0)

In [14]:
selected_words_model = tc.logistic_classifier.create(train_data,target='sentiment', features=selected_words, validation_set=test_data)
sentiment_model = tc.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 2         | 11       | 2.669978  | 0.950669     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 1.103070     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 1.341517     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 2.117359     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [28]:
selected_words_model.coefficients.sort('value').print_rows()

+-----------+-------+-------+-----------------------+----------------------+
|    name   | index | class |         value         |        stderr        |
+-----------+-------+-------+-----------------------+----------------------+
|  horrible |  None |   1   |   -2.251335236759098  | 0.08020249388788436  |
|  terrible |  None |   1   |   -2.223661436085132  | 0.07731736203785752  |
|   awful   |  None |   1   |  -2.0529082040313567  | 0.10099735435259254  |
|    hate   |  None |   1   |  -1.3484407222463166  | 0.07715698604297333  |
|    bad    |  None |   1   |  -0.9914778800650621  | 0.038484286646990606 |
|    wow    |  None |   1   | -0.009538236067684405 |  0.1604641122471165  |
|   great   |  None |   1   |   0.8630655001196581  | 0.018955052444376858 |
| fantastic |  None |   1   |   0.8858047568814243  |  0.1116759129339967  |
|  amazing  |  None |   1   |   1.1000933113660234  | 0.09954776260465989  |
|  awesome  |  None |   1   |   1.1335346660341385  |  0.0839964398318753  |

In [16]:
selected_words_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.336591384887764,0.008929969787656441
awesome,None,1,1.1335346660341385,0.0839964398318753
great,None,1,0.8630655001196581,0.018955052444376858
fantastic,None,1,0.8858047568814243,0.1116759129339967
amazing,None,1,1.1000933113660234,0.09954776260465989
love,None,1,1.3592688669225095,0.028068300152099074
horrible,None,1,-2.251335236759098,0.08020249388788436
bad,None,1,-0.9914778800650621,0.038484286646990606
terrible,None,1,-2.223661436085132,0.07731736203785752
awful,None,1,-2.0529082040313567,0.10099735435259254


In [18]:
evaluate_selected = selected_words_model.evaluate(test_data)
evaluate_original = sentiment_model.evaluate(test_data)

In [19]:
print(evaluate_selected)

{'accuracy': 0.8463848186404036, 'auc': 0.6935096220934976, 'confusion_matrix': Columns:
	target_label	int
	predicted_label	int
	count	int

Rows: 4

Data:
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|      1       |        0        |  159  |
|      0       |        0        |  371  |
|      0       |        1        |  4957 |
|      1       |        1        | 27817 |
+--------------+-----------------+-------+
[4 rows x 3 columns]
, 'f1_score': 0.9157860082304526, 'log_loss': 0.39622654670874075, 'precision': 0.8487520595594068, 'recall': 0.9943165570488991, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int

Rows: 1001

Data:
+-----------+--------------------+-----+-------+------+
| threshold |        fpr         | tpr |   p   |  n   |
+-----------+--------------------+-----+-------+------+
|    0.0    |        1.0         | 1.0 | 27976 | 5328 |
|   0.001   | 0.9994369369369369

In [20]:
print(evaluate_original)

{'accuracy': 0.9176975738650012, 'auc': 0.9258242975424673, 'confusion_matrix': Columns:
	target_label	int
	predicted_label	int
	count	int

Rows: 4

Data:
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|      0       |        1        |  1397 |
|      1       |        0        |  1344 |
|      0       |        0        |  3931 |
|      1       |        1        | 26632 |
+--------------+-----------------+-------+
[4 rows x 3 columns]
, 'f1_score': 0.951057941255245, 'log_loss': 0.3304787187240578, 'precision': 0.9501587641371436, 'recall': 0.9519588218472976, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int

Rows: 1001

Data:
+-----------+--------------------+--------------------+-------+------+
| threshold |        fpr         |        tpr         |   p   |  n   |
+-----------+--------------------+--------------------+-------+------+
|    0.0    |        1.0         |        1.0

In [22]:
products['predicted_sentiment_original'] = sentiment_model.predict(products, output_type = 'probability')
products['predicted_sentiment_selected'] = selected_words_model.predict(products, output_type = 'probability')

In [23]:
giraffe_reviews = products[products['name']== 'Vulli Sophie the Giraffe Teether']
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment_original', ascending=False)
print(giraffe_reviews)
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment_selected', ascending=False)
print(giraffe_reviews)

+-------------------------------+-------------------------------+--------+-----------+
|              name             |             review            | rating | sentiment |
+-------------------------------+-------------------------------+--------+-----------+
| Vulli Sophie the Giraffe T... | Sophie, oh Sophie, your ti... |  5.0   |     1     |
| Vulli Sophie the Giraffe T... | I'll be honest...I bought ... |  4.0   |     1     |
| Vulli Sophie the Giraffe T... | As a mother of 16month old... |  5.0   |     1     |
| Vulli Sophie the Giraffe T... | We got this little giraffe... |  5.0   |     1     |
| Vulli Sophie the Giraffe T... | As every mom knows, you al... |  5.0   |     1     |
| Vulli Sophie the Giraffe T... | My Mom-in-Law bought Sophi... |  5.0   |     1     |
| Vulli Sophie the Giraffe T... | My 4 month old son is teet... |  4.0   |     1     |
| Vulli Sophie the Giraffe T... | Let me just start off by a... |  5.0   |     1     |
| Vulli Sophie the Giraffe T... | I'm not s

In [25]:
evaluate_giraffe_selected = selected_words_model.evaluate(giraffe_reviews)
evaluate_giraffe_original = sentiment_model.evaluate(giraffe_reviews)

In [26]:
print(evaluate_giraffe_selected)

{'accuracy': 0.8755186721991701, 'auc': 0.6612903225806451, 'confusion_matrix': Columns:
	target_label	int
	predicted_label	int
	count	int

Rows: 4

Data:
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|      0       |        1        |   87  |
|      1       |        1        |  627  |
|      0       |        0        |   6   |
|      1       |        0        |   3   |
+--------------+-----------------+-------+
[4 rows x 3 columns]
, 'f1_score': 0.9330357142857142, 'log_loss': 0.3647694138336622, 'precision': 0.8781512605042017, 'recall': 0.9952380952380953, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int

Rows: 1001

Data:
+-----------+-----+-----+-----+----+
| threshold | fpr | tpr |  p  | n  |
+-----------+-----+-----+-----+----+
|    0.0    | 1.0 | 1.0 | 630 | 93 |
|   0.001   | 1.0 | 1.0 | 630 | 93 |
|   0.002   | 1.0 | 1.0 | 630 | 93 |
|   0.003   | 1.0 | 1.0 | 630 | 93 

In [27]:
print(evaluate_giraffe_original)

{'accuracy': 0.975103734439834, 'auc': 0.9860556408943507, 'confusion_matrix': Columns:
	target_label	int
	predicted_label	int
	count	int

Rows: 4

Data:
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|      0       |        0        |   80  |
|      1       |        1        |  625  |
|      1       |        0        |   5   |
|      0       |        1        |   13  |
+--------------+-----------------+-------+
[4 rows x 3 columns]
, 'f1_score': 0.9858044164037855, 'log_loss': 0.08680353643340637, 'precision': 0.9796238244514106, 'recall': 0.9920634920634921, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int

Rows: 1001

Data:
+-----------+--------------------+--------------------+-----+----+
| threshold |        fpr         |        tpr         |  p  | n  |
+-----------+--------------------+--------------------+-----+----+
|    0.0    |        1.0         |        1.0         | 

In [30]:
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment_original', ascending=False)
print(diaper_champ_reviews[0])
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment_selected', ascending=False)
print(diaper_champ_reviews[0])

{'name': 'Baby Trend Diaper Champ', 'review': "I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out.", 'rating': 4.0, 'sentiment': 1, 'word_count': {'key': 1.0, 'have': 1.0, 'pieces': 1.0, 'between': 1.0, 'cushioned': 1.0, 'piece': 1.0, 'take': 1.0, 'overloading': 1.0, 'be': 1.0, 'looking': 1.0, 're': 1.0, 'stinkiness': 1.0, 'retain': 1.0, 'now': 1.0, 'wonderful': 1.0, 'worked': 1.0, '16': 1.0, 'and': 3.0, 'months': 1.0, 've': 1.0, 'in': 1.0, 'us': 1.0, 'i': 3.0, 'experienced': 1.0, 'read': 1.0, 'easy': 1.0, 'for': 3.0